<a href="https://colab.research.google.com/github/kithmini-wijesiri/Binding-Affinity-Prediction-with-ML-based-Docking/blob/main/ML_%26_Active_Learning_for_Molecular_Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Molecular docking to predict the binding pose and affinity of small molecules in a protein binding site


The epidermal growth factor receptor \(EGFR\) will be used as our target system \([6VHN](https://www.rcsb.org/structure/6VHN)\).
The protein encoded by this gene is a transmembrane glycoprotein that is a member of the protein kinase superfamily.
 are going to try to predict the binding affinity of some molecules to this protein to the ATP binding site in the hinge region

EGFR is a frequently over-expressed and aberrantly activated trans-membrane protein in non-small cell lung cancer (NSCLC) patients, described for the first time in 2004. Mutations in this gene are associated with lung cancer in particular.

The goal of this project is twofold:
- First, design a molecular docking workflow using the classical docking software Autodock Vina.
- Second, build an active learning / Bayesian optimization cycle to find high binding affinity candidates from a molecular screening library while minimizing the number of evaluations of the \(somewhat expensive\) Docking oracle.

Such pipelines could be used in the hit\-to\-lead phase of the drug\-discovery process or to reduce the size of the screening library to be tested.

These active learning pipelines can be used in other phase of the drug\-discovery pipelines like lead-optimization or to select the best candidates for experimental/wet lab validation.

<img src="https://drive.google.com/uc?export=view&id=1uIhRXkYFBMFB6i6XImHfXAa35gYEx7YL" width="500"/>

Fig. : Rendering (VMD) of EGFR (6VHN) in complex with inhibitor.


### Molecular docking

In the modern drug discovery pipeline, determining the binding mode and binding affinity of an active molecule to a given protein target is very important. With the advent of on\-demand synthesis libraries such as [Enamine Real](https://enamine.net/compound-collections/real-compounds/real-database), the design space of possible ligands extends to billions of theoretically synthesizable molecules. Molecular docking can probe and \(de\)prioritize these molecules before they are even synthesized, thus, accelerate the discovery of novel lead candidates.

Molecular docking software can be used to predict binding modes and affinities by sampling possible conformations of a ligand inside the protein binding pocket \(Fig. 1\). To this end, the sampling of conformations aims to optimize a Docking score function, which is assumed to increase  After the optimal pose has been found, which is typically calculated from a variety of terms for different non\-covalent molecular interactions; e.g. electrostatics and van der Waals energies.

<img src="https://drive.google.com/uc?export=view&id=1kakPkA9wf8XzkidYXRx16zYD3qubjf4h" width="500"/>

**Fig. 1:** EGFR in complex with inhibitor



### Background: Binding free energy

We are interested in finding drug compounds that strongly bind to a target protein.
Hence, in the equilibrium reaction between protein $P$ and ligand $L$
\begin{equation}
 P + L 	\rightleftharpoons PL\ ,
\end{equation}
we are interested in a large ratio of protein-ligand complexes over protein and ligand individually.

<img src="https://drive.google.com/uc?export=view&id=1-D-GcRNUrmLKlVsSv40O5NXHtxyBrNj2" width="500"/>

**Fig. 2:** Protein-ligand binding equilibrium reaction

This ratio is proportional to the binding constant $K_b^0$, which on the other hand, is proportional to the Gibbs free energy of binding
\begin{equation}
 \Delta G^0_\mathrm{bind} = -k_B T \ln K_b^0 \ ,
\end{equation}
where $k_B$ is the Boltzmann constant and $T$ is the temperature. Hence, we are interested in predicting $ \Delta G^0_\mathrm{bind}$ in-silico. While $ \Delta G^0_\mathrm{bind}$ can be computed physically rigorous via MD simulations, this is too computationally expensive for large-scale screenings in the hit-to-lead phase. To this end, we can use Docking, providing a less accurate estimate of $ \Delta G^0_\mathrm{bind}$ using significantly less compute in order to (de-)prioritize compounds.



### Docking score

An empirical mathematical function used to __proxy__ the binding free energy of a protein-ligand complex. Used to rank ligands in a virtual screening by considering
the inter- & intra molecular interactions and other physical properties (H-bonds, hydrophobic interactions, etc.) to define a scalar value.

The docking score c in [AutoDock Vina](https://onlinelibrary.wiley.com/doi/10.1002/jcc.21334) is built of pairwise physics\-inspired functions f
\begin{equation}
c = \sum_{i<j} f_{t_it_j}(r_{ij}) \ ,
\end{equation}
where $t_i$ is the atom type and $r_{ij}$ is the pairwise distance between atoms $i$ and $j$.
The Docking score consists of both intra and inter\-molecular interactions
\begin{equation}
c =c_\mathrm{inter} + c_\mathrm{intra} \ ,
\end{equation}
but only the inter\-molecular component is used to predict the binding free energy:
\begin{equation}
 \Delta G^0_\mathrm{bind} \approx g(c_\mathrm{inter}) = \frac{c_\mathrm{inter}}{1+w N_\mathrm{rot}} \ ,
\end{equation}
where $N_\mathrm{rot}$ is the number of rotable bonds of the ligand and $w$ is a weight parameter.
The parameters of $c$ are optimized based on the PDBBind database.



### Sampling algorithms

To sample the conformations of ligands inside a protein binding pocket, classical Docking algorithms employ sampling algorithms such as the following:

* **Matching algorithms:** Compare the shape similarity of ligand conformations and the protein binding pocket, usually also including chemical information, e.g. hydrogen bond acceptors and donors. However, these programs require a prior computation of ligand conformations that are used during shape comparison.This will fail if the biologically relevant conformation is not present in this library.

* In the **incremental instruction** method, the ligand is first deconstructed into smaller fragments by breaking its rotatable bonds. One of the fragments, for example the biggest one, is placed first into the binding pocket. Subsequently, the complete ligand is incrementally constructed inside the binding pocket by connecting the remaining fragments at the appropriate positions of the core fragment.

* **Monte Carlo methods** sample ligand conformations by rigid\-body rotation and translation as well as bond rotation. They generate random placements and evaluate obtained conformations inside the protein binding pocket with the pose score function. If the pose is accepted, the conformation is saved and subsequently randomly modified to generate another conformations.

* **Genetic algorithms**

Vina uses a genetic algorithm for global search and optimizes local minima using the BFGS quasi-Newton optimizer.



A **molecular docking** workflow usually involves the following steps \(Fig. 3\):

- Input file preparation, e.g. protonation and conversion into specific file formats
- Conformational sampling of the ligand inside the binding pocket via classical optimization of the docking score
- Scoring of the generated docking poses via and docking score $c$ and prediction of $\Delta G$ via $g(c_\mathrm{inter})$
- Post\-processing, e.g. visualization of the docking pose

<img src="https://drive.google.com/uc?export=view&id=1-otcUrfJkfkJVZgsxvKb85EFHIJMdquP" width="500"/>

__Fig. 3:__ Molecular docking workflow



## Implementation

The implementation consists of 2 parts.

 First, we will prepare the Protein and compound database for Docking and deomonstrate Docking of a single compound.
 Second part, we will use this pipeline to dock batches of compounds and actively learn a Gaussian Process surrogate model of the docking score for efficient screening.

### Input preparation

In this workshop we will consider the simplest case of rigid docking with a known binding site. The information of the binding site will be provided by the crystallized ligand in the PDB entry.

In real case scenario, the binding site can be unknown, in this case the research of the binding site can be done through various techniques or be achieved through blind docking. Recently, ML methods \(i.e. DiffDock\) have shown great potential in this task.



### Limitations

* Docking programs can consider some residue sidechains flexible during docking calculations to account for binding pocket flexibility. However, the dynamic, adaptive nature of the protein\-ligand binding and the contribution of the conformational entropy of the protein is insufficiently considered by protein\-ligand docking. This can result in false positives: Even if the ligand finds a suitable pose in the binding pocket, this position is not guaranteed until the protein is allowed to explore near\-minima conformations. Hence, short molecular dynamics \(MD\) simulations are recommended to evaluate the stability of the predicted binding pose.

* Scoring functions used by docking programs must be cheap to compute. While the accuracy is good enough to distinguish good poses from bad poses, it can have problems sorting the best poses. For example, while most popular docking programs are able to find the experimental pose in their calculations, this pose is rarely the best one of the proposed set. Furthermore, several retrospective studies have shown that docking scores often poorly correlate with binding affinity.

* While blind docking is possible, in order to reduce computational cost, docking is often only performed on a subset of the protein \(typically around a known binding pocket\). Choosing the correct binding site is a challenge, if the binding pocket is not known a priori.

* To maximize the accuracy of the calculation, the ligand and protein structures must be prepared appropriately. Protonation states of amino acids and the ligands can be tricky to get right, especially in the case of \(potential\) tautomers. This introduces yet another cause to obtain inaccurate results.

Due to these limitations, Docking predictions tend to be not very accurate with a mean absolute error of 2.85 kcal/mol on PDBBind  [\(Vina\)](https://onlinelibrary.wiley.com/doi/10.1002/jcc.21334).

<img src="https://drive.google.com/uc?export=view&id=1OP8tn0xzPOI4w6a_x_mm8DWBCV_rELNo" width="500"/>

### Preparing a pdb

For the docking, the pdb file of the protein and the ligand needs to be prepared.
Protein preparation workflow:

- Removed the ligand from the pdb file
- Deleted all the water molecules/solvent from the pdb file
- Converted residues to standard residues  
- Completed sydechains
- Added hydrogens to the protein to the correct protonation state (ph 7.4)
- Added charges to the protein (Gasteiger model)
- Changed names of the residues to AMBER ff14Sb names

## Discussion

Classical docking software relies on hand\-crafted, physics\-inspired functional forms as well as classical optimization routines to find docking poses and predict binding affinity. Several ML directions aim to improve on this:

- DiffDock trains a diffusion model to predict the binding pose to improve the accuracy of the pose and aims to speed\-up the prediction by avoiding the optimization scheme.
- Several ML models learn scoring functions using the PDBBind database to improve the accuracy of the binding affinity prediction.
- AlphaFold3 co\-folds the protein with the ligand within the binding site, reducing the need of high\-quality protein holo crystal structure, which is assumed to be known by Docking approaches.

The efficiency of the active learning loop could be improved via more sophisticated batch active learning approaches or by building more sophisticated surrogate models.

#### Physically rigorous approaches using MD

- Short outlook to free energy simulations
- Connection to neural network potentials

#### Improving the active learning loop

The active learning loop and model we used could be improved in several ways:

1. ##### Binding Affinity Prediction:
    
We used a simple gaussian process with a RBF kernel trained on fingerprints to predict the binding affinity.
More sophisticated models could be used to improve the prediction accuracy.

Binding Affinity is a complex property that depends on the protein-ligand interaction,

the solvation energy,
the entropy of the system, and the enthalpy of the system and our model
is not even taking in consideration the protein structure.
    
Using some 3D descriptors like Smooth Overlaps of Atomistic Position or Atomic Cluster Expansion could improve the prediction.

2. ##### Active Learning:
    
We used a quite simple uncertainty estimation (max Uncertainty), but we can use more sophisticated acquisition functions to guide the optimization process.
In this way we can balance our efforts of exploration and exploitation.

Some of the more commons acquisition functions are:

- ##### Expected Improvement (EI)

<center><<img src="https://drive.google.com/uc?export=view&id=19K-CvdpTPRvcG_-DE8Xsy7st3Otg1eK3" width="1200"/>/></center>

$$ PI(x) = \psi\Big( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \Big),$$

- ##### Probability of Improvement (PI)

<center><<img src="https://drive.google.com/uc?export=view&id=1Bk4obCSSvvchudiLHrSUC-y2mi0ul05M" width="1200"/>/></center>

$$ \begin{split}\begin{align*}
EI(x) = & (\mu(x) - f(x^+) - \xi) \psi\Big( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \Big) \\
& + \sigma(x) \phi\Big( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \Big),
\end{align*}\end{split} $$

- ##### Upper Confidence Bound (UCB)

<center><<img src="https://drive.google.com/uc?export=view&id=1sJ3Sgxf7uQ_DXBI6xJwNqEWQAKJv9DWX" width="1200"/>/></center>


$$ UCB(x) = \mu(x) + \beta \sigma(x), $$
